In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from typing import List
from tqdm import tqdm

%matplotlib inline

import catalyst 
import recbole

from typing import Dict, List, Tuple

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.nn.init import constant_, xavier_normal_
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset

from catalyst import dl, metrics
from catalyst.contrib.datasets import MovieLens
from catalyst.utils import get_device, set_global_seed
from torch.nn.utils.rnn import pad_sequence 

import random

from recbole.model.abstract_recommender import SequentialRecommender
from recbole.model.layers import *


set_global_seed(100)

### ML_100k

In [ ]:
df = pd.read_csv('data/ML_100k.csv')
df = df.rename(columns={'userId': 'user_id', 'movieId': 'item_id'})
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')
df['weekday'] = pd.to_datetime(df.timestamp).dt.weekday
df['hour'] = pd.to_datetime(df.timestamp).dt.hour
df.head()

,user_id,item_id,rating,timestamp,weekday,hour
0,1,31,2.5,2009-12-14 02:52:24,0,2
1,1,1029,3.0,2009-12-14 02:52:59,0,2
2,1,1061,3.0,2009-12-14 02:53:02,0,2
3,1,1129,2.0,2009-12-14 02:53:05,0,2
4,1,1172,4.0,2009-12-14 02:53:25,0,2


In [ ]:
splitter = RandomSplit(test_fraction=0.2)
train_df, valid_df, test_df = splitter(df)

In [ ]:
train_grouped = train_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id, 
                                                                 x.timestamp,
                                                                 x.weekday,
                                                                 x.hour), key=lambda x: x[1])]
).reset_index()
train_grouped.rename({0:'train_interactions'}, axis=1, inplace=True)

valid_grouped = valid_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id,
                                                         x.timestamp,
                                                         x.weekday,
                                                         x.hour), key=lambda x: x[1])]
).reset_index()
valid_grouped.rename({0:'valid_interactions'}, axis=1, inplace=True)

test_grouped = test_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id,
                                                         x.timestamp,
                                                         x.weekday,
                                                         x.hour), key=lambda x: x[1])]
).reset_index()
test_grouped.rename({0:'test_interactions'}, axis=1, inplace=True)


train_grouped.head()

,user_id,train_interactions
0,1,"[(2294, 2009-12-14 02:51:48, 0, 2), (2455, 200..."
1,2,"[(150, 1996-06-21 11:09:55, 4, 11), (296, 1996..."
2,3,"[(355, 2011-02-28 02:53:09, 0, 2), (1271, 2011..."
3,4,"[(1210, 2000-02-05 19:25:14, 5, 19), (2734, 20..."
4,5,"[(1380, 2006-11-12 23:10:44, 6, 23), (1035, 20..."


In [ ]:
joined = train_grouped.merge(valid_grouped).merge(test_grouped)
joined.head()

,user_id,train_interactions,valid_interactions,test_interactions
0,1,"[(2294, 2009-12-14 02:51:48, 0, 2), (2455, 200...","[(1287, 2009-12-14 02:53:07, 0, 2), (1953, 200...","[(2193, 2009-12-14 02:53:18, 0, 2), (2968, 200..."
1,2,"[(150, 1996-06-21 11:09:55, 4, 11), (296, 1996...","[(551, 1996-06-21 11:16:07, 4, 11), (273, 1996...","[(319, 1996-06-21 11:18:38, 4, 11), (485, 1996..."
2,3,"[(355, 2011-02-28 02:53:09, 0, 2), (1271, 2011...","[(2028, 2011-02-28 19:37:42, 0, 19), (110, 201...","[(1721, 2011-02-28 20:00:36, 0, 20), (377, 201..."
3,4,"[(1210, 2000-02-05 19:25:14, 5, 19), (2734, 20...","[(3169, 2000-02-07 10:21:44, 0, 10), (1298, 20...","[(1858, 2000-02-07 10:35:38, 0, 10), (3108, 20..."
4,5,"[(1380, 2006-11-12 23:10:44, 6, 23), (1035, 20...","[(1968, 2006-11-12 23:32:04, 6, 23), (30707, 2...","[(33679, 2006-11-12 23:35:17, 6, 23), (5995, 2..."


In [ ]:
our_items = set()
for idx, row in tqdm(joined.iterrows()):
    for el in row.train_interactions:
        our_items.add(el[0])
        
len(our_items)

671it [00:00, 22009.34it/s]


6811

In [ ]:
item2idx = {k: i for i, k in enumerate(our_items)}
idx2item = {i: k for k, i in item2idx.items()}

In [ ]:
class MyDataset(Dataset):
    
    def __init__(self, ds, num_items, item2idx, phase='valid', N=200):
        super().__init__()
        self.ds = ds
        self.phase = phase
        self.n_items = num_items
        self.item2idx = item2idx
        self.N = N 
        
    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self, idx):
        
        row = self.ds.iloc[idx]
        
        x_input = np.zeros(self.n_items+1)
        x_input[[self.item2idx[x[0]]+1 for x in row['train_interactions'] if x[0] in self.item2idx]] = 1
        
        days_of_weeks = [x[2] for x in row['train_interactions'] if x[0] in self.item2idx][-self.N+1:]
        
        hours = [x[3] for x in row['train_interactions'] if x[0] in self.item2idx][-self.N+1:]
        
        seq_input = [self.item2idx[x[0]]+1 for x in row['train_interactions'] if x[0] in self.item2idx][-self.N+1:]
        
        targets = np.zeros(self.n_items+1)
        
        dow_valid = row['valid_interactions'][0][2]
        dow_test = row['test_interactions'][0][2]
        
        hours_valid = row['valid_interactions'][0][3]
        hours_test = row['test_interactions'][0][3]
        
        if self.phase == 'train':
            return (seq_input, days_of_weeks, hours, dow_valid, hours_valid)
        elif self.phase == 'valid':
            targets[[self.item2idx[x[0]]+1 for x in row['valid_interactions'] if x[0] in self.item2idx]] = 1
        else:
            return (seq_input, days_of_weeks, hours, dow_test, hours_test)
            
        return (targets, seq_input, days_of_weeks, hours, dow_valid, hours_valid)


671 671


In [ ]:
n_items = len(item2idx)

train = MyDataset(ds=joined,
                  num_items=n_items, 
                  item2idx=item2idx,
                  phase='train')

valid = MyDataset(ds=joined,
                  num_items=n_items,
                  item2idx=item2idx,
                  phase='valid')

print(len(train),len(valid))

In [ ]:
def collate_fn_train(batch: List[Tuple[torch.Tensor]]) -> Dict[str, torch.Tensor]: 
    
    seq_i,days_of_weeks,hours,dow_valid,hours_valid = zip(*batch)
    seq_len = torch.Tensor([len(x) for x in seq_i])
    dow_valid = torch.Tensor([x for x in dow_valid])
    hours_valid = torch.Tensor([x for x in hours_valid])
    seq_i = pad_sequence([torch.Tensor(t) for t in seq_i]).T    
    days_of_weeks = pad_sequence([torch.Tensor(t) for t in days_of_weeks]).T
    hours = pad_sequence([torch.Tensor(t) for t in hours]).T
    
    return {'seq_i': seq_i, 
            'seq_len':seq_len,
            'dow': days_of_weeks,
            'hours': hours,
            'dow_valid': dow_valid,
            'hours_valid': hours_valid}


def collate_fn_valid(batch: List[Tuple[torch.Tensor]]) -> Dict[str, torch.Tensor]:
    
    y, seq_i, days_of_weeks, hours, dow_valid, hours_valid = zip(*batch)
    
    seq_len = torch.Tensor([len(x) for x in seq_i]).long()
    seq_i = pad_sequence([torch.Tensor(t) for t in seq_i]).T.long()
    days_of_weeks = pad_sequence([torch.Tensor(t) for t in days_of_weeks]).T.long()
    hours = pad_sequence([torch.Tensor(t) for t in hours]).T.long()
    dow_valid = torch.Tensor([x for x in dow_valid])
    hours_valid = torch.Tensor([x for x in hours_valid])
            
    targets = pad_sequence([torch.Tensor(t) for t in y]).T

    return {"targets": targets,
            'seq_i': seq_i,
            'seq_len':seq_len,
            'dow': days_of_weeks,
            'hours': hours,
            'dow_valid': dow_valid,
            'hours_valid': hours_valid}

In [ ]:
loaders = {
        "train": DataLoader(train, batch_size=256, collate_fn=collate_fn_train),
        "valid": DataLoader(valid, batch_size=256, collate_fn=collate_fn_valid),
}

In [ ]:

model = BERT4Rec(n_items=len(item2idx)+1, mask_ratio=0.2, hidden_size=32)

optimizer = optim.Adam(model.parameters(), lr=0.001)
lr_scheduler = StepLR(optimizer, step_size=20, gamma=0.1)
engine = dl.DeviceEngine('cpu')
hparams = {
    "anneal_cap": 0.2,
    "total_anneal_steps": 6000,
}


callbacks = [
    dl.NDCGCallback("logits", "targets", [20]),
    dl.MAPCallback("logits", "targets", [10]),
    dl.OptimizerCallback("loss", accumulation_steps=1),
    dl.EarlyStoppingCallback(
        patience=7, loader_key="valid", metric_key="map10", minimize=False
    )
]


runner = RecSysRunner()
runner.train(
    model=model,
    optimizer=optimizer,
    engine=engine,
    hparams=hparams,
    scheduler=lr_scheduler,
    loaders=loaders,
    num_epochs=100,
    verbose=True,
    timeit=True,
    callbacks=callbacks,
)


/Users/annakomarova/anaconda3/lib/python3.10/site-packages/catalyst/core/runner.py:637: UserWarning: No ``ISchedulerCallback/SchedulerCallback`` were found while runner.scheduler is not None.Do you make scheduler step during ``runner.handle_batch``?
  warnings.warn(
1/100 * Epoch (train): 100%|██████████| 3/3 [00:08<00:00,  2.97s/it, _timer/_fps=57.961, _timer/batch_time=2.743, _timer/data_time=2.004, _timer/model_time=0.739, loss=8.801, lr=1.000e-03, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00] 


train (1/100) loss: 8.817609209950385 | lr: 0.001 | map10: 0.0 | map10/std: 0.0 | momentum: 0.9 | ndcg20: 0.0 | ndcg20/std: 0.0


1/100 * Epoch (valid): 100%|██████████| 3/3 [00:05<00:00,  1.78s/it, _timer/_fps=100.001, _timer/batch_time=1.590, _timer/data_time=0.025, _timer/model_time=1.565, loss=8.768, lr=1.000e-03, map10=0.015, momentum=0.900, ndcg20=0.006]


valid (1/100) loss: 8.753207117185863 | lr: 0.001 | map10: 0.021590022464600653 | map10/std: 0.007997730888560109 | momentum: 0.9 | ndcg20: 0.007608442112731285 | ndcg20/std: 0.002752989317687319
* Epoch (1/100) 


2/100 * Epoch (train): 100%|██████████| 3/3 [00:08<00:00,  2.90s/it, _timer/_fps=61.439, _timer/batch_time=2.588, _timer/data_time=1.747, _timer/model_time=0.841, loss=8.729, lr=1.000e-03, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00] 


...

valid (30/100) loss: 7.650187534298165 | lr: 0.001 | map10: 0.11585952035094516 | map10/std: 0.017758133705273457 | momentum: 0.9 | ndcg20: 0.07079297593959931 | ndcg20/std: 0.005553716875240385
* Epoch (30/100) 


31/100 * Epoch (train): 100%|██████████| 3/3 [00:12<00:00,  4.04s/it, _timer/_fps=40.987, _timer/batch_time=3.879, _timer/data_time=2.874, _timer/model_time=1.006, loss=7.661, lr=1.000e-03, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00] 


train (31/100) loss: 7.662603519712582 | lr: 0.001 | map10: 0.0 | map10/std: 0.0 | momentum: 0.9 | ndcg20: 0.0 | ndcg20/std: 0.0


31/100 * Epoch (valid): 100%|██████████| 3/3 [00:05<00:00,  1.71s/it, _timer/_fps=132.248, _timer/batch_time=1.202, _timer/data_time=0.025, _timer/model_time=1.177, loss=7.640, lr=1.000e-03, map10=0.159, momentum=0.900, ndcg20=0.086]

valid (31/100) loss: 7.636357813407341 | lr: 0.001 | map10: 0.12341881009310084 | map10/std: 0.020148011938828616 | momentum: 0.9 | ndcg20: 0.07631984004409409 | ndcg20/std: 0.005762211191972171
* Epoch (31/100) 


In [ ]:
test_runner = RecSysRunner(model=model)

In [ ]:
test_dataset = MyDataset(ds=joined, num_items=n_items, phase='test',item2idx=item2idx)


inference_loader = DataLoader(test_dataset, 
                              batch_size=joined.shape[0]//100, 
                              collate_fn=collate_fn_train,)

preds = []

for prediction in tqdm(runner.predict_loader(loader=inference_loader)):
    preds.extend(prediction.detach().cpu().numpy().tolist())
    
print(len(preds))
assert len(preds) == joined.shape[0]

joined['preds_bert4rec'] = preds
joined['recs_bert4rec_10'] = joined['preds_bert4rec'].apply(lambda x: np.argsort(-np.array(x))[:10])
joined['recs_bert4rec_10'] = joined['recs_bert4rec_10'].apply(lambda x: [idx2item[t-1] for t in x])
joined['recs_bert4rec_5'] = joined['preds_bert4rec'].apply(lambda x: np.argsort(-np.array(x))[:5])
joined['recs_bert4rec_5'] = joined['recs_bert4rec_5'].apply(lambda x: [idx2item[t-1] for t in x])
joined.drop(['preds_bert4rec'],axis=1, inplace=True)
joined.head()

112it [00:08, 13.18it/s]


671


,user_id,train_interactions,valid_interactions,test_interactions,recs_bert4rec_10,recs_bert4rec_5
0,1,"[(2294, 2009-12-14 02:51:48, 0, 2), (2455, 200...","[(1287, 2009-12-14 02:53:07, 0, 2), (1953, 200...","[(2193, 2009-12-14 02:53:18, 0, 2), (2968, 200...","[593, 356, 296, 260, 480, 2571, 318, 110, 1270...","[593, 356, 296, 260, 480]"
1,2,"[(150, 1996-06-21 11:09:55, 4, 11), (296, 1996...","[(551, 1996-06-21 11:16:07, 4, 11), (273, 1996...","[(319, 1996-06-21 11:18:38, 4, 11), (485, 1996...","[260, 2571, 318, 1270, 2959, 608, 1196, 1, 344...","[260, 2571, 318, 1270, 2959]"
2,3,"[(355, 2011-02-28 02:53:09, 0, 2), (1271, 2011...","[(2028, 2011-02-28 19:37:42, 0, 19), (110, 201...","[(1721, 2011-02-28 20:00:36, 0, 20), (377, 201...","[260, 480, 2571, 110, 1270, 457, 47, 608, 1196...","[260, 480, 2571, 110, 1270]"
3,4,"[(1210, 2000-02-05 19:25:14, 5, 19), (2734, 20...","[(3169, 2000-02-07 10:21:44, 0, 10), (1298, 20...","[(1858, 2000-02-07 10:35:38, 0, 10), (3108, 20...","[593, 356, 2571, 318, 110, 2959, 457, 1, 608, 47]","[593, 356, 2571, 318, 110]"
4,5,"[(1380, 2006-11-12 23:10:44, 6, 23), (1035, 20...","[(1968, 2006-11-12 23:32:04, 6, 23), (30707, 2...","[(33679, 2006-11-12 23:35:17, 6, 23), (5995, 2...","[593, 296, 260, 480, 2571, 318, 110, 1270, 457...","[593, 296, 260, 480, 2571]"


In [ ]:
evaluate_recommender(joined, model_preds='recs_bert4rec_10')

{'ndcg': 0.1280307170558313, 'recall': 0.03206971653677037}

In [ ]:
evaluate_recommender(joined, model_preds='recs_bert4rec_5')

{'ndcg': 0.08535465507261165, 'recall': 0.019260518398816713}

### ML-1m

In [ ]:
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
df = pd.read_table('data/ratings.dat', sep='::',header=None, names=rnames, engine='python')
df = df.rename(columns={'userId': 'user_id', 'movie_id': 'item_id'})
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')
df['weekday'] = pd.to_datetime(df.timestamp).dt.weekday
df['hour'] = pd.to_datetime(df.timestamp).dt.hour
df.head()

,user_id,item_id,rating,timestamp,weekday,hour
0,1,1193,5,2000-12-31 22:12:40,6,22
1,1,661,3,2000-12-31 22:35:09,6,22
2,1,914,3,2000-12-31 22:32:48,6,22
3,1,3408,4,2000-12-31 22:04:35,6,22
4,1,2355,5,2001-01-06 23:38:11,5,23


In [ ]:
splitter = RandomSplit(test_fraction=0.2)
train_df, valid_df, test_df = splitter(df)

In [ ]:
train_grouped = train_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id, 
                                                                 x.timestamp,
                                                                 x.weekday,
                                                                 x.hour), key=lambda x: x[1])]
).reset_index()
train_grouped.rename({0:'train_interactions'}, axis=1, inplace=True)

valid_grouped = valid_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id,
                                                         x.timestamp,
                                                         x.weekday,
                                                         x.hour), key=lambda x: x[1])]
).reset_index()
valid_grouped.rename({0:'valid_interactions'}, axis=1, inplace=True)

test_grouped = test_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id,
                                                         x.timestamp,
                                                         x.weekday,
                                                         x.hour), key=lambda x: x[1])]
).reset_index()
test_grouped.rename({0:'test_interactions'}, axis=1, inplace=True)


train_grouped.head()

,user_id,train_interactions
0,1,"[(3186, 2000-12-31 22:00:19, 6, 22), (1270, 20..."
1,2,"[(1198, 2000-12-31 21:28:44, 6, 21), (1210, 20..."
2,3,"[(593, 2000-12-31 21:10:18, 6, 21), (2858, 200..."
3,4,"[(1210, 2000-12-31 20:18:44, 6, 20), (1097, 20..."
4,5,"[(2717, 2000-12-31 05:37:52, 6, 5), (908, 2000..."


In [ ]:
joined = train_grouped.merge(valid_grouped).merge(test_grouped)
joined.head()

,user_id,train_interactions,valid_interactions,test_interactions
0,1,"[(3186, 2000-12-31 22:00:19, 6, 22), (1270, 20...","[(2791, 2000-12-31 22:36:28, 6, 22), (2321, 20...","[(2687, 2001-01-06 23:37:48, 5, 23), (745, 200..."
1,2,"[(1198, 2000-12-31 21:28:44, 6, 21), (1210, 20...","[(2028, 2000-12-31 21:56:13, 6, 21), (2571, 20...","[(1372, 2000-12-31 21:59:01, 6, 21), (1552, 20..."
2,3,"[(593, 2000-12-31 21:10:18, 6, 21), (2858, 200...","[(648, 2000-12-31 21:24:27, 6, 21), (2735, 200...","[(1270, 2000-12-31 21:30:31, 6, 21), (1079, 20..."
3,4,"[(1210, 2000-12-31 20:18:44, 6, 20), (1097, 20...","[(2947, 2000-12-31 20:23:50, 6, 20), (1214, 20...","[(1240, 2000-12-31 20:24:20, 6, 20), (2951, 20..."
4,5,"[(2717, 2000-12-31 05:37:52, 6, 5), (908, 2000...","[(2323, 2000-12-31 06:50:45, 6, 6), (272, 2000...","[(1715, 2000-12-31 06:58:11, 6, 6), (1653, 200..."


In [ ]:
our_items = set()
for idx, row in tqdm(joined.iterrows()):
    for el in row.train_interactions:
        our_items.add(el[0])
        
len(our_items)

6040it [00:00, 40652.52it/s]


3636

In [ ]:
item2idx = {k: i for i, k in enumerate(our_items)}
idx2item = {i: k for k, i in item2idx.items()}

In [ ]:
n_items = len(item2idx)

train = MyDataset(ds=joined,
                  num_items=n_items, 
                  item2idx=item2idx,
                  phase='train')

valid = MyDataset(ds=joined,
                  num_items=n_items,
                  item2idx=item2idx,
                  phase='valid')

print(len(train),len(valid))

6040 6040


In [ ]:
loaders = {
        "train": DataLoader(train, batch_size=256, collate_fn=collate_fn_train),
        "valid": DataLoader(valid, batch_size=256, collate_fn=collate_fn_valid),
}

In [ ]:

model = BERT4Rec(n_items=len(item2idx)+1, mask_ratio=0.2, hidden_size=32)

optimizer = optim.Adam(model.parameters(), lr=0.001)
lr_scheduler = StepLR(optimizer, step_size=20, gamma=0.1)
engine = dl.DeviceEngine('cpu')
hparams = {
    "anneal_cap": 0.2,
    "total_anneal_steps": 6000,
}


callbacks = [
    dl.NDCGCallback("logits", "targets", [20]),
    dl.MAPCallback("logits", "targets", [10]),
    dl.OptimizerCallback("loss", accumulation_steps=1),
    dl.EarlyStoppingCallback(
        patience=7, loader_key="valid", metric_key="map10", minimize=False
    )
]


runner = RecSysRunner()
runner.train(
    model=model,
    optimizer=optimizer,
    engine=engine,
    hparams=hparams,
    scheduler=lr_scheduler,
    loaders=loaders,
    num_epochs=100,
    verbose=True,
    timeit=True,
    callbacks=callbacks,
)


/Users/annakomarova/anaconda3/lib/python3.10/site-packages/catalyst/core/runner.py:637: UserWarning: No ``ISchedulerCallback/SchedulerCallback`` were found while runner.scheduler is not None.Do you make scheduler step during ``runner.handle_batch``?
  warnings.warn(
1/100 * Epoch (train): 100%|██████████| 24/24 [01:27<00:00,  3.64s/it, _timer/_fps=69.996, _timer/batch_time=2.172, _timer/data_time=1.459, _timer/model_time=0.712, loss=7.722, lr=1.000e-03, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00] 


train (1/100) loss: 7.968992110119752 | lr: 0.001 | map10: 0.0 | map10/std: 0.0 | momentum: 0.9 | ndcg20: 0.0 | ndcg20/std: 0.0


1/100 * Epoch (valid): 100%|██████████| 24/24 [00:56<00:00,  2.35s/it, _timer/_fps=90.903, _timer/batch_time=1.672, _timer/data_time=0.035, _timer/model_time=1.637, loss=7.689, lr=1.000e-03, map10=0.099, momentum=0.900, ndcg20=0.061] 


valid (1/100) loss: 7.696078093951901 | lr: 0.001 | map10: 0.11778197297394671 | map10/std: 0.014863163885628503 | momentum: 0.9 | ndcg20: 0.0633254435036751 | ndcg20/std: 0.0070999549995383925
* Epoch (1/100) 


2/100 * Epoch (train): 100%|██████████| 24/24 [01:23<00:00,  3.46s/it, _timer/_fps=74.131, _timer/batch_time=2.050, _timer/data_time=1.369, _timer/model_time=0.681, loss=7.427, lr=1.000e-03, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00] 


...

valid (54/100) loss: 6.115810972807423 | lr: 0.001 | map10: 0.23737514068354046 | map10/std: 0.023480422168979775 | momentum: 0.9 | ndcg20: 0.15402954491163723 | ndcg20/std: 0.017437280896189143
* Epoch (54/100) 


55/100 * Epoch (train): 100%|██████████| 24/24 [00:53<00:00,  2.25s/it, _timer/_fps=93.128, _timer/batch_time=1.632, _timer/data_time=0.913, _timer/model_time=0.720, loss=6.220, lr=1.000e-03, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00] 


train (55/100) loss: 6.2903915954741425 | lr: 0.001 | map10: 0.0 | map10/std: 0.0 | momentum: 0.9 | ndcg20: 0.0 | ndcg20/std: 0.0


55/100 * Epoch (valid): 100%|██████████| 24/24 [00:50<00:00,  2.11s/it, _timer/_fps=157.895, _timer/batch_time=0.963, _timer/data_time=0.037, _timer/model_time=0.926, loss=6.047, lr=1.000e-03, map10=0.239, momentum=0.900, ndcg20=0.152]

valid (55/100) loss: 6.105169727628594 | lr: 0.001 | map10: 0.23468514505206356 | map10/std: 0.021876374812646388 | momentum: 0.9 | ndcg20: 0.15346711534538016 | ndcg20/std: 0.016831209036916248
* Epoch (55/100) 


In [ ]:
test_runner = RecSysRunner(model=model)

In [ ]:
test_dataset = MyDataset(ds=joined, num_items=n_items, phase='test',item2idx=item2idx)


inference_loader = DataLoader(test_dataset, 
                              batch_size=joined.shape[0]//100, 
                              collate_fn=collate_fn_train,)

preds = []

for prediction in tqdm(runner.predict_loader(loader=inference_loader)):
    preds.extend(prediction.detach().cpu().numpy().tolist())
    
print(len(preds))
assert len(preds) == joined.shape[0]

joined['preds_bert4rec'] = preds
joined['recs_bert4rec_10'] = joined['preds_bert4rec'].apply(lambda x: np.argsort(-np.array(x))[:10])
joined['recs_bert4rec_10'] = joined['recs_bert4rec_10'].apply(lambda x: [idx2item[t-1] for t in x])
joined['recs_bert4rec_5'] = joined['preds_bert4rec'].apply(lambda x: np.argsort(-np.array(x))[:5])
joined['recs_bert4rec_5'] = joined['recs_bert4rec_5'].apply(lambda x: [idx2item[t-1] for t in x])
joined.drop(['preds_bert4rec'],axis=1, inplace=True)
joined.head()

101it [00:31,  3.19it/s]


6040


,user_id,train_interactions,valid_interactions,test_interactions,recs_bert4rec,recs_bert4rec_10,recs_bert4rec_5
0,1,"[(3186, 2000-12-31 22:00:19, 6, 22), (1270, 20...","[(2791, 2000-12-31 22:36:28, 6, 22), (2321, 20...","[(2687, 2001-01-06 23:37:48, 5, 23), (745, 200...","[2396, 593, 527, 1265, 34, 1617, 2858, 318, 1,...","[2396, 593, 527, 1265, 34, 1617, 2858, 318, 1,...","[2396, 593, 527, 1265, 34]"
1,2,"[(1198, 2000-12-31 21:28:44, 6, 21), (1210, 20...","[(2028, 2000-12-31 21:56:13, 6, 21), (2571, 20...","[(1372, 2000-12-31 21:59:01, 6, 21), (1552, 20...","[2289, 1641, 1394, 2918, 34, 2028, 1079, 1197,...","[2289, 1641, 1394, 2918, 34, 2028, 1079, 1197,...","[2289, 1641, 1394, 2918, 34]"
2,3,"[(593, 2000-12-31 21:10:18, 6, 21), (2858, 200...","[(648, 2000-12-31 21:24:27, 6, 21), (2735, 200...","[(1270, 2000-12-31 21:30:31, 6, 21), (1079, 20...","[10, 316, 329, 380, 780, 1527, 349, 1552, 165,...","[10, 316, 329, 380, 780, 1527, 349, 1552, 165,...","[10, 316, 329, 380, 780]"
3,4,"[(1210, 2000-12-31 20:18:44, 6, 20), (1097, 20...","[(2947, 2000-12-31 20:23:50, 6, 20), (1214, 20...","[(1240, 2000-12-31 20:24:20, 6, 20), (2951, 20...","[589, 1240, 2571, 1214, 1200, 541, 110, 3703, ...","[589, 1240, 2571, 1214, 1200, 541, 110, 3703, ...","[589, 1240, 2571, 1214, 1200]"
4,5,"[(2717, 2000-12-31 05:37:52, 6, 5), (908, 2000...","[(2323, 2000-12-31 06:50:45, 6, 6), (272, 2000...","[(1715, 2000-12-31 06:58:11, 6, 6), (1653, 200...","[25, 337, 1358, 778, 300, 1885, 1682, 529, 117...","[25, 337, 1358, 778, 300, 1885, 1682, 529, 117...","[25, 337, 1358, 778, 300]"


In [ ]:
evaluate_recommender(joined, model_preds='recs_bert4rec_10')

{'ndcg': 0.14581013099020618, 'recall': 0.0337442523686209}

In [ ]:
evaluate_recommender(joined, model_preds='recs_bert4rec_5')

{'ndcg': 0.08740025855788641, 'recall': 0.01756498438783866}

### ML-20m

In [11]:
df = pd.read_csv('ML_20m.csv')
df = df.rename(columns={'userId': 'user_id', 'movieId': 'item_id'})
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')
df['weekday'] = pd.to_datetime(df.timestamp).dt.weekday
df['hour'] = pd.to_datetime(df.timestamp).dt.hour
df.head()

,user_id,item_id,rating,timestamp,weekday,hour
0,1,2,3.5,2005-04-02 23:53:47,5,23
1,1,29,3.5,2005-04-02 23:31:16,5,23
2,1,32,3.5,2005-04-02 23:33:39,5,23
3,1,47,3.5,2005-04-02 23:32:07,5,23
4,1,50,3.5,2005-04-02 23:29:40,5,23


In [12]:
splitter = RandomSplit(test_fraction=0.2)
train_df, valid_df, test_df = splitter(df)

In [13]:
train_grouped = train_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id, 
                                                                 x.timestamp,
                                                                 x.weekday,
                                                                 x.hour), key=lambda x: x[1])]
).reset_index()
train_grouped.rename({0:'train_interactions'}, axis=1, inplace=True)

valid_grouped = valid_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id,
                                                         x.timestamp,
                                                         x.weekday,
                                                         x.hour), key=lambda x: x[1])]
).reset_index()
valid_grouped.rename({0:'valid_interactions'}, axis=1, inplace=True)

test_grouped = test_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id,
                                                         x.timestamp,
                                                         x.weekday,
                                                         x.hour), key=lambda x: x[1])]
).reset_index()
test_grouped.rename({0:'test_interactions'}, axis=1, inplace=True)


train_grouped.head()

,user_id,train_interactions
0,1,"[(924, 2004-09-10 03:06:38, 4, 3), (919, 2004-..."
1,2,"[(62, 2000-11-21 15:29:58, 1, 15), (469, 2000-..."
2,3,"[(589, 1999-12-11 07:25:08, 5, 7), (1188, 1999..."
3,4,"[(380, 1996-08-24 09:27:05, 5, 9), (165, 1996-..."
4,5,"[(17, 1996-12-25 15:15:35, 2, 15), (62, 1996-1..."


In [14]:
joined = train_grouped.merge(valid_grouped).merge(test_grouped)
joined.head()

,user_id,train_interactions,valid_interactions,test_interactions
0,1,"[(924, 2004-09-10 03:06:38, 4, 3), (919, 2004-...","[(2947, 2005-04-02 23:46:20, 5, 23), (8961, 20...","[(7164, 2005-04-02 23:52:03, 5, 23), (2021, 20..."
1,2,"[(62, 2000-11-21 15:29:58, 1, 15), (469, 2000-...","[(3926, 2000-11-21 15:34:49, 1, 15), (1973, 20...","[(1969, 2000-11-21 15:36:09, 1, 15), (1970, 20..."
2,3,"[(589, 1999-12-11 07:25:08, 5, 7), (1188, 1999...","[(3070, 1999-12-11 13:42:09, 5, 13), (2872, 19...","[(1356, 1999-12-14 12:51:10, 1, 12), (1603, 19..."
3,4,"[(380, 1996-08-24 09:27:05, 5, 9), (165, 1996-...","[(370, 1996-08-24 09:34:03, 5, 9), (594, 1996-...","[(596, 1996-08-24 09:37:04, 5, 9), (531, 1996-..."
4,5,"[(17, 1996-12-25 15:15:35, 2, 15), (62, 1996-1...","[(720, 1996-12-25 15:27:30, 2, 15), (350, 1996...","[(376, 1996-12-26 16:24:34, 3, 16), (1079, 199..."


In [15]:
our_items = set()
for idx, row in tqdm(joined.iterrows()):
    for el in row.train_interactions:
        our_items.add(el[0])
        
len(our_items)

35378it [00:02, 15430.55it/s]


15736

In [16]:
item2idx = {k: i for i, k in enumerate(our_items)}
idx2item = {i: k for k, i in item2idx.items()}

In [19]:
n_items = len(item2idx)

train = MyDataset(ds=joined,
                  num_items=n_items, 
                  item2idx=item2idx,
                  phase='train')

valid = MyDataset(ds=joined,
                  num_items=n_items,
                  item2idx=item2idx,
                  phase='valid')

print(len(train),len(valid))

35378 35378


In [22]:
loaders = {
        "train": DataLoader(train, batch_size=256, collate_fn=collate_fn_train),
        "valid": DataLoader(valid, batch_size=256, collate_fn=collate_fn_valid),
}

In [23]:

model = BERT4Rec(n_items=len(item2idx)+1, mask_ratio=0.2, hidden_size=32)

optimizer = optim.Adam(model.parameters(), lr=0.001)
lr_scheduler = StepLR(optimizer, step_size=20, gamma=0.1)
engine = dl.DeviceEngine('cpu')
hparams = {
    "anneal_cap": 0.2,
    "total_anneal_steps": 6000,
}


callbacks = [
    dl.NDCGCallback("logits", "targets", [20]),
    dl.MAPCallback("logits", "targets", [10]),
    dl.OptimizerCallback("loss", accumulation_steps=1),
    dl.EarlyStoppingCallback(
        patience=7, loader_key="valid", metric_key="map10", minimize=False
    )
]


runner = RecSysRunner()
runner.train(
    model=model,
    optimizer=optimizer,
    engine=engine,
    hparams=hparams,
    scheduler=lr_scheduler,
    loaders=loaders,
    num_epochs=100,
    verbose=True,
    timeit=True,
    callbacks=callbacks,
)


1/100 * Epoch (train):   0%|          | 0/139 [00:00<?, ?it/s]

train (1/100) loss: 8.199406338800133 | lr: 0.001 | map10: 0.0 | map10/std: 0.0 | momentum: 0.9 | ndcg20: 0.0 | ndcg20/std: 0.0


1/100 * Epoch (valid):   0%|          | 0/139 [00:00<?, ?it/s]

valid (1/100) loss: 7.682598683312328 | lr: 0.001 | map10: 0.14191976870354386 | map10/std: 0.01575361833242672 | momentum: 0.9 | ndcg20: 0.08572437047985326 | ndcg20/std: 0.008670231579484424
* Epoch (1/100) 


2/100 * Epoch (train):   0%|          | 0/139 [00:00<?, ?it/s]

...

38/100 * Epoch (valid):   0%|          | 0/139 [00:00<?, ?it/s]

valid (38/100) loss: 6.14646630435702 | lr: 0.001 | map10: 0.22083035571207613 | map10/std: 0.0183051990267605 | momentum: 0.9 | ndcg20: 0.16575575793090702 | ndcg20/std: 0.013995228678057867
* Epoch (38/100) 


39/100 * Epoch (train):   0%|          | 0/139 [00:00<?, ?it/s]

Keyboard Interrupt


KeyboardInterrupt: ignored

In [43]:
test_dataset = MyDataset(ds=joined, num_items=n_items, phase='test',item2idx=item2idx)


inference_loader = DataLoader(test_dataset, 
                              batch_size=joined.shape[0]//100, 
                              collate_fn=collate_fn_train,)

preds = []

for prediction in tqdm(runner.predict_loader(loader=inference_loader, engine=dl.DeviceEngine('cpu'))):
    preds.extend(prediction.detach().cpu().numpy().tolist())
    
print(len(preds))
assert len(preds) == joined.shape[0]

joined['preds_bert4rec'] = preds
joined['recs_bert4rec_10'] = joined['preds_bert4rec'].apply(lambda x: np.argsort(-np.array(x))[:10])
joined['recs_bert4rec_10'] = joined['recs_bert4rec_10'].apply(lambda x: [idx2item[t-1] for t in x])
joined['recs_bert4rec_5'] = joined['preds_bert4rec'].apply(lambda x: np.argsort(-np.array(x))[:5])
joined['recs_bert4rec_5'] = joined['recs_bert4rec_5'].apply(lambda x: [idx2item[t-1] for t in x])
joined.drop(['preds_bert4rec'],axis=1, inplace=True)
joined.head()

101it [00:54,  1.86it/s]


35378


,user_id,train_interactions,valid_interactions,test_interactions,recs_bert4rec_10,recs_bert4rec_5
0,1,"[(924, 2004-09-10 03:06:38, 4, 3), (919, 2004-...","[(2947, 2005-04-02 23:46:20, 5, 23), (8961, 20...","[(7164, 2005-04-02 23:52:03, 5, 23), (2021, 20...","[1197, 1961, 1097, 1270, 1265, 1394, 1220, 608...","[1197, 1961, 1097, 1270, 1265]"
1,2,"[(62, 2000-11-21 15:29:58, 1, 15), (469, 2000-...","[(3926, 2000-11-21 15:34:49, 1, 15), (1973, 20...","[(1969, 2000-11-21 15:36:09, 1, 15), (1970, 20...","[2369, 2428, 2719, 2694, 2606, 3826, 2605, 299...","[2369, 2428, 2719, 2694, 2606]"
2,3,"[(589, 1999-12-11 07:25:08, 5, 7), (1188, 1999...","[(3070, 1999-12-11 13:42:09, 5, 13), (2872, 19...","[(1356, 1999-12-14 12:51:10, 1, 12), (1603, 19...","[1129, 2640, 1375, 3702, 2001, 2406, 2021, 264...","[1129, 2640, 1375, 3702, 2001]"
3,4,"[(380, 1996-08-24 09:27:05, 5, 9), (165, 1996-...","[(370, 1996-08-24 09:34:03, 5, 9), (594, 1996-...","[(596, 1996-08-24 09:37:04, 5, 9), (531, 1996-...","[95, 553, 442, 474, 2, 141, 62, 235, 11, 17]","[95, 553, 442, 474, 2]"
4,5,"[(17, 1996-12-25 15:15:35, 2, 15), (62, 1996-1...","[(720, 1996-12-25 15:27:30, 2, 15), (350, 1996...","[(376, 1996-12-26 16:24:34, 3, 16), (1079, 199...","[474, 348, 515, 508, 555, 357, 265, 272, 377, ...","[474, 348, 515, 508, 555]"


In [44]:
evaluate_recommender(joined, model_preds='recs_bert4rec_10')

{'ndcg': 0.16356511797620038, 'recall': 0.04399354663059405}

In [45]:
evaluate_recommender(joined, model_preds='recs_bert4rec_5')

{'ndcg': 0.09440482277692609, 'recall': 0.021120154268991623}